In [1]:
import json
import os
from pyquaternion import Quaternion
import pandas as pd
import pickle
import numpy as np
import math

from utils.data_dirs import data_dirs
from utils.get_camera_heading import get_camera_heading
from utils.get_camera_position import get_camera_position
from utils.get_camera_rotation import get_camera_rotation
from utils.get_heading_from_north import get_heading_from_north
from utils.get_heading import get_heading
from utils.index import index
from utils.normalize_angle import normalize_angle
from utils.unique import unique

In [2]:
EXPERIMENT = False

In [3]:
base_dir, output_dir, folder, EXPERIMENT_DATA, suffix, *_ = data_dirs(EXPERIMENT)
print(base_dir)
print(output_dir)
print(folder)
print(EXPERIMENT_DATA)
print(suffix)

/Users/chanwutk/Documents/apperception/data/v1.0-mini
/Users/chanwutk/Documents/apperception/data/v1.0-mini
v1.0-mini
/work/apperception/data/raw/scenic/experiment_data



In [4]:
with open(os.path.join(base_dir, folder, 'calibrated_sensor.json')) as f:
    calibrated_sensor_json = json.load(f)

with open(os.path.join(base_dir, folder, 'category.json')) as f:
    category_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sample.json')) as f:
    sample_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sample_data.json')) as f:
    sample_data_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sample_annotation.json')) as f:
    sample_annotation_json = json.load(f)

with open(os.path.join(base_dir, folder, 'instance.json')) as f:
    instance_json = json.load(f)

with open(os.path.join(base_dir, folder, 'scene.json')) as f:
    scene_json = json.load(f)

with open(os.path.join(base_dir, folder, 'ego_pose.json')) as f:
    ego_pose_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sensor.json')) as f:
    sensor_json = json.load(f)

In [5]:
if EXPERIMENT:
    files = os.listdir(EXPERIMENT_DATA)
    files_set = set(files)
    sample_data_filter = [s for s in sample_data_json if s['filename'].split('/')[2] in files_set]
else:
    sample_data_filter = sample_data_json
sample_data_filter = [s for s in sample_data_filter if s['fileformat'] == 'jpg']

In [6]:
sample_data_filter[0]

{'token': '512015c209c1490f906982c3b182c2a8',
 'sample_token': '39586f9d59004284a7114a68825e8eec',
 'ego_pose_token': '512015c209c1490f906982c3b182c2a8',
 'calibrated_sensor_token': '1d31c729b073425e8e0202c5c6e66ee1',
 'timestamp': 1532402927762460,
 'fileformat': 'jpg',
 'is_key_frame': False,
 'height': 900,
 'width': 1600,
 'filename': 'sweeps/CAM_FRONT/n015-2018-07-24-11-22-45+0800__CAM_FRONT__1532402927762460.jpg',
 'prev': '68e8e98cf7b0487baa139df808641db7',
 'next': 'e9df123af5b747adbd9ddf6454e1abdc'}

In [7]:
sample_tokens = unique(sample_data_filter, 'sample_token')
sample_filter = [
    {
        'sample_token': s['token'],
        'scene_token': s['scene_token']
    }
    for s in sample_json
    if s['token'] in sample_tokens
]
len(sample_filter)

404

In [8]:
calibrated_sensor_tokens = unique(sample_data_filter, 'calibrated_sensor_token')
calibrated_sensor_filter = [
    {
        'calibrated_sensor_token': c['token'],
        'camera_translation': c['translation'],
        'camera_rotation': c['rotation'],
        'camera_intrinsic': c['camera_intrinsic']
    }
    for c in calibrated_sensor_json
    if c['token'] in calibrated_sensor_tokens
]
len(calibrated_sensor_filter)

60

In [9]:
ego_pose_tokens = unique(sample_data_filter, 'ego_pose_token')
ego_pose_filter = [
    {
        'ego_pose_token': e['token'],
        'ego_translation': e['translation'],
        'ego_rotation': e['rotation']
    }
    for e in ego_pose_json
    if e['token'] in ego_pose_tokens
]
len(ego_pose_filter)

14008

In [10]:
scene_tokens = unique(sample_filter, 'scene_token')
scene_filter = [
    {
        'scene_token': s['token'],
        'scene_name': s['name'],
    }
    for s in scene_json
    if s['token'] in scene_tokens
]
len(scene_filter)

10

In [11]:
sample_map = index(sample_filter, 'sample_token')
calibrated_sensor_map = index(calibrated_sensor_filter, 'calibrated_sensor_token')
ego_pose_map = index(ego_pose_filter, 'ego_pose_token')
scene_map = index(scene_filter, 'scene_token')

def s_map(s):
    sample = sample_map[s['sample_token']]
    calibrated_sensor = calibrated_sensor_map[s['calibrated_sensor_token']]
    ego_pose = ego_pose_map[s['ego_pose_token']]
    scene = scene_map[sample['scene_token']]
    
    ego_heading = get_heading_from_north(Quaternion(ego_pose['ego_rotation']))
    camera_heading = get_camera_heading(Quaternion(calibrated_sensor['camera_rotation']))
    ret = {
        **s,
        **sample,
        **calibrated_sensor,
        **ego_pose,
        **scene,
        'ego_heading': ego_heading * 180 / math.pi,
        'camera_heading': normalize_angle(camera_heading + ego_heading) * 180 / math.pi,
        'camera_translation': get_camera_position(
            calibrated_sensor['camera_translation'],
            ego_pose['ego_translation'],
            ego_pose['ego_rotation'],
        ),
        'camera_rotation': get_camera_rotation(
            calibrated_sensor['camera_rotation'],
            ego_pose['ego_rotation'],
        ),
    }
    del ret['ego_pose_token']
    del ret['calibrated_sensor_token']
    del ret['fileformat']
    del ret['height']
    del ret['width']
    del ret['prev']
    del ret['next']
    del ret['scene_token']
    return ret

sample_data_res = [*map(s_map, sample_data_filter)]

len(sample_data_res)

14008

In [12]:
sample_data_res[0]

{'token': '512015c209c1490f906982c3b182c2a8',
 'sample_token': '39586f9d59004284a7114a68825e8eec',
 'timestamp': 1532402927762460,
 'is_key_frame': False,
 'filename': 'sweeps/CAM_FRONT/n015-2018-07-24-11-22-45+0800__CAM_FRONT__1532402927762460.jpg',
 'camera_translation': array([ 410.40065095, 1178.27549786,    1.48668958]),
 'camera_rotation': array([-0.12828446,  0.1140136 ,  0.70480565, -0.68832623]),
 'camera_intrinsic': [[1266.417203046554, 0.0, 816.2670197447984],
  [0.0, 1266.417203046554, 491.50706579294757],
  [0.0, 0.0, 1.0]],
 'ego_translation': [410.9433704785875, 1179.9098624203461, 0.0],
 'ego_rotation': [0.5731318069963492,
  -0.001008743951274553,
  0.013420038692137808,
  -0.8193526815762804],
 'scene_name': 'scene-0061',
 'ego_heading': 159.93489949860546,
 'camera_heading': 160.26062025345016}

In [13]:
sample_annotation_filter = [
    sa
    for sa in sample_annotation_json
    if sa['sample_token'] in sample_tokens
]
len(sample_annotation_filter)

18538

In [14]:
instance_tokens = unique(sample_annotation_filter, 'instance_token')
instance_filter = [
    {
        'instance_token': i['token'],
        'category_token': i['category_token']
    }
    for i in instance_json
    if i['token'] in instance_tokens
]
len(instance_filter)

911

In [15]:
category_tokens = unique(instance_filter, 'category_token')
category_filter = [
    {
        'category_token': c['token'],
        'category': c['name']
    }
    for c in category_json
    if c['token'] in category_tokens
]
len(category_filter)

18

In [16]:
instance_map = index(instance_filter, 'instance_token')
category_map = index(category_filter, 'category_token')

def sa_map(sa):
    instance = instance_map[sa['instance_token']]
    ret = {
        **sa,
        **instance,
        **category_map[instance['category_token']],
        'heading': (get_heading_from_north(Quaternion(sa['rotation']))) * 180 / math.pi
    }
    
    del ret['visibility_token']
    del ret['attribute_tokens']
    del ret['prev']
    del ret['next']
    del ret['num_lidar_pts']
    del ret['num_radar_pts']
    del ret['category_token']
    
    return ret

sample_annotation_res = [*map(sa_map, sample_annotation_filter)]
len(sample_annotation_res)

18538

In [17]:
sample_data_final = pd.DataFrame(sample_data_res)
sample_data_final[:1]

,token,sample_token,timestamp,is_key_frame,filename,camera_translation,camera_rotation,camera_intrinsic,ego_translation,ego_rotation,scene_name,ego_heading,camera_heading
0,512015c209c1490f906982c3b182c2a8,39586f9d59004284a7114a68825e8eec,1532402927762460,False,sweeps/CAM_FRONT/n015-2018-07-24-11-22-45+0800...,"[410.40065095332744, 1178.2754978561568, 1.486...","[-0.1282844630496619, 0.11401360185255394, 0.7...","[[1266.417203046554, 0.0, 816.2670197447984], ...","[410.9433704785875, 1179.9098624203461, 0.0]","[0.5731318069963492, -0.001008743951274553, 0....",scene-0061,159.934899,160.26062


In [18]:
sample_annotation_final = pd.DataFrame(sample_annotation_res)
sample_annotation_final[:1]

,token,sample_token,instance_token,translation,size,rotation,category,heading
0,70aecbe9b64f4722ab3c230391a3beb8,cd21dbfc3bd749c7b10a5c42562e0c42,6dd2cbf4c24b4caeb625035869bca7b5,"[373.214, 1130.48, 1.25]","[0.621, 0.669, 1.642]","[0.9831098797903927, 0.0, 0.0, -0.183016295062...",human.pedestrian.adult,-111.091


In [19]:
len(sample_annotation_final)

18538

In [20]:
df_sample_data_keyframe = sample_data_final[sample_data_final["is_key_frame"]][
    ["token", "sample_token"]
]
sample_annotation_final = sample_annotation_final.set_index("sample_token").join(
    df_sample_data_keyframe.set_index("sample_token"), on="sample_token", rsuffix="_sample_data"
)
sample_annotation_final[:1]

,token,instance_token,translation,size,rotation,category,heading,token_sample_data
sample_token,,,,,,,,
cd21dbfc3bd749c7b10a5c42562e0c42,70aecbe9b64f4722ab3c230391a3beb8,6dd2cbf4c24b4caeb625035869bca7b5,"[373.214, 1130.48, 1.25]","[0.621, 0.669, 1.642]","[0.9831098797903927, 0.0, 0.0, -0.183016295062...",human.pedestrian.adult,-111.091,e0bc3ff06fe348fdb56039c640be0ac3


In [21]:
len(sample_annotation_final)

111228

In [22]:
len({*df_sample_data_keyframe["token"].tolist()})

2424

In [23]:
len({*df_sample_data_keyframe["sample_token"].tolist()})

404

In [24]:
sample_data_final = sample_data_final.sort_values(by=['scene_name', 'timestamp'])

In [25]:
sample_data_final["frame_order"] = 0
scene = ""
i = 1
for index, row in sample_data_final.iterrows():
    if row['scene_name'] != scene:
        scene = row['scene_name']
        sample_data_final.loc[index, "frame_order"] = 1
        i = 2
    else:
        sample_data_final.loc[index, "frame_order"] = i
        i += 1


In [26]:
print(len(sample_data_final))

14008


In [27]:
print(len(sample_annotation_final))

111228


In [28]:
with open(os.path.join(output_dir, f"sample_data{suffix}.pickle"), "wb") as f:
    pickle.dump(sample_data_final, f)
with open(os.path.join(output_dir, f"annotation{suffix}.pickle"), "wb") as f:
    pickle.dump(sample_annotation_final, f)

In [29]:
sample_data_final.to_csv(os.path.join(output_dir, f"sample_data{suffix}.csv"), index=False)

In [30]:
sample_annotation_final.to_csv(os.path.join(output_dir, f"annotation{suffix}.csv"), index=False)